<a href="https://colab.research.google.com/github/usintec/BitcoinStockMarket/blob/master/RegressionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install needed packages (optional in Colab)

In [ ]:
!pip install --quiet seaborn statsmodels

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
url = 'https://raw.githubusercontent.com/usintec/BitcoinStockMarket/refs/heads/master/bitcoin_ngx_data_template_csv.csv'
df = pd.read_csv(url)

df.head()

# Drop rows with missing values

In [ ]:

df = df[['BTC_Volatility', 'NGX_Volatility', 'Regulation']].dropna()

STEP 3: Create interaction term

In [ ]:
df['Interaction'] = df['BTC_Volatility'] * df['Regulation']

STEP 4: Fit the moderated regression model

In [ ]:
model = smf.ols('NGX_Volatility ~ BTC_Volatility + Regulation + Interaction', data=df).fit()

Display regression summary

In [ ]:
print(model.summary())


STEP 5: Interpretation of β3 (Interaction Term)

In [ ]:
beta3 = model.params['Interaction']
p_value = model.pvalues['Interaction']
alpha = 0.05

In [ ]:
print("\n--- Moderation Effect Interpretation ---")
print(f"β3 (Interaction Term) = {beta3:.4f}")
print(f"p-value = {p_value:.4f}")

In [ ]:
if p_value < alpha:
    direction = "weakens" if beta3 < 0 else "amplifies"
    print(f"✅ Significant moderation detected: Regulation {direction} the spillover effect of BTC on NGX.")
else:
    print("❌ No significant moderation: Regulation does not alter the BTC→NGX relationship.")

In [ ]:
# STEP 6: Visualization
plt.figure(figsize=(10, 6))
sns.lmplot(data=df, x='BTC_Volatility', y='NGX_Volatility', hue='Regulation',
           palette='Set1', height=6, aspect=1.5, markers=["o", "s"])
plt.title("Moderation: BTC Volatility vs NGX Volatility by Regulation")
plt.xlabel("Bitcoin Volatility")
plt.ylabel("NGX Volatility")
plt.grid(True)
plt.tight_layout()
plt.show()